In [1]:
import torch
from torch import nn
import torch
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
import torch.optim as optim
import argparse


import numpy as np
import pandas as pd

Model

In [2]:
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        #Initialize two convolutional layers in ResNet block
        self.block1 = nn.Sequential (
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=0),
            nn.BatchNorm2d(out_channels)
        )
        #Skip connection
        self.skip_connection = nn.Identity()
        if in_channels != out_channels or stride != 1:
            self.skip_connection = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=1, bias=0),
                nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        identity_x = x.skip_connection
        x = self.block1(x)
        x = x + identity_x
        return x


In [3]:
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=0),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = ResNetBlock(3, 64, stride=1)
        self.act1 = nn.ReLU()
        self.layer2 = ResNetBlock(64, 128, stride=2)
        self.act2 = nn.ReLU()
        self.layer3 = ResNetBlock(128, 256, stride=2)
        self.act3 = nn.ReLU()
        self.layer4 = ResNetBlock(256, 512, stride=2)
        self.act4 = nn.ReLU()
        self.fc1 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.act4(self.layer4(x))
        x = self.fc1(x)
        return x

In [4]:
resnet = ResNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0005)

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.RandomCrop(size=32, padding=4),
     transforms.RandomHorizontalFlip(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

Data Loader

In [6]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=True, num_workers=2)

100%|██████████| 170498071/170498071 [00:10<00:00, 15761354.17it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
def train_network (resnet, n_epochs):
    resnet.to(device)
    for epoch in range(n_epochs):
        train_loss = 0
        correct = 0
        total = 0

        for i, data in enumerate(trainloader, 0):
            inputs, labels = data

            inputs, labels = inputs.to(device), labels.to(device)

            #zero the parameter gradient
            optimizer.zero_grad()

            #forward + backward + optimize
            output = resnet(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = output.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            print(i, len(trainloader), 'Per-batch Training Loss: %.3f | Acc: %.3f%%'
                     % (train_loss/(i+1), 100.0*correct/total))

    print('Model Training Accuracy: %.3f' % 100.0*correct/total)